In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import time


In [2]:
# 브라우저 꺼짐 방지 옵션
chrome_options = Options()
chrome_options.headless=True
chrome_options.add_experimental_option("detach", True)


In [39]:
def scroll_down(driver):
    last_element = None  
    while True:
        url_raw = driver.find_elements(By.CSS_SELECTOR, '.JobCard_JobCard__Tb7pI a') 
        if not url_raw: 
            break
        if last_element == url_raw[-1]:  
            return url_raw
        last_element = url_raw[-1]  
        ActionChains(driver).move_to_element(last_element).perform()
        time.sleep(1)  

In [40]:
driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1900, 1000)
# 웹페이지가 로드될 때까지 2초를 대기
driver.implicitly_wait(time_to_wait=2)  

# 원티드드 사이트로 이동합니다.
driver.get(url='https://www.wanted.co.kr/wdlist?country=kr&job_sort=job.latest_order&years=0&locations=all')
#직군 선택 
driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div[1]/section/div[1]/section/div/button/span/span[2]').click()
driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div[1]/section/div[1]/section/div/div/div[2]/div[2]/ul[1]/button[2]').click()
develope_oc = WebDriverWait(driver, 3).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "SelectItem_SelectItem__IcYKH"))
)
Occupation_list={'종료':0}
num=2
for button in develope_oc:
    try:
        occupation = WebDriverWait(button, 3).until(
            EC.presence_of_element_located((By.CLASS_NAME, "wds-1c3y3mo"))
        )
        Occupation_list[occupation.text]=num
        num+=1
    except:
        pass
print(Occupation_list)
time.sleep(1)
while True:
    input_occupation=input('직군선택: ')
    if input_occupation == '종료':
        break
    occupation=Occupation_list[input_occupation]
    driver.find_element(By.XPATH, f'//*[@id="__next"]/div[3]/div[1]/section/div[1]/section/div/div/div[2]/div[2]/ul[2]/button[{occupation}]/span').click()
driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div[1]/section/div[1]/section/div/div/div[2]/div[3]/button[2]/span[2]').click()
time.sleep(1)
scroll_down(driver)
url_raw=scroll_down(driver)
url_list=[]
for url in url_raw:
    try:
        link = url.get_attribute("href")  
        url_list.append(link)
    except:
        pass
print(len(url_list))



{'종료': 0, '소프트웨어 엔지니어': 2, '서버 개발자': 3, '프론트엔드 개발자': 4, '웹 개발자': 5, '자바 개발자': 6, '파이썬 개발자': 7, '머신러닝 엔지니어': 8, 'C,C++ 개발자': 9, '데이터 엔지니어': 10, 'DevOps / 시스템 관리자': 11, 'Node.js 개발자': 12, '안드로이드 개발자': 13, '임베디드 개발자': 14, '시스템,네트워크 관리자': 15, 'QA,테스트 엔지니어': 16, '개발 매니저': 17, '보안 엔지니어': 18, 'iOS 개발자': 19, '데이터 사이언티스트': 20, '기술지원': 21, '하드웨어 엔지니어': 22, '프로덕트 매니저': 23, '빅데이터 엔지니어': 24, '웹 퍼블리셔': 25, '.NET 개발자': 26, '크로스플랫폼 앱 개발자': 27, 'PHP 개발자': 28, '블록체인 플랫폼 엔지니어': 29, '영상,음성 엔지니어': 30, 'DBA': 31, 'ERP전문가': 32, '그래픽스 엔지니어': 33, 'CTO,Chief Technology Officer': 34, '루비온레일즈 개발자': 35, 'BI 엔지니어': 36, 'VR 엔지니어': 37, 'CIO,Chief Information Officer': 38, 'RPA 엔지니어': 39, '테크니컬 라이터': 40}
60


In [41]:
len(url_raw)

60